In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import time
from matplotlib import pyplot as plt

## 1 数据导入和预览

### 1.1 训练集和测试集导入

In [2]:
train_data = pd.read_excel("data/BankChurners.xlsx",sheet_name=0)
test_data = pd.read_excel("data/BankChurners.xlsx",sheet_name=1)
train_data.head()

,ID,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,1,Existing Customer,43,F,4,High School,Unknown,Less than $40K,Blue,39,...,3,3,2786.0,1793,993.0,0.803,3646,68,0.659,0.644
1,2,Existing Customer,54,M,4,Graduate,Married,$120K +,Blue,50,...,2,0,2872.0,2035,837.0,0.613,1770,47,0.741,0.709
2,3,Attrited Customer,49,F,3,High School,Married,Less than $40K,Blue,45,...,2,3,2951.0,2437,514.0,0.765,2519,36,0.565,0.826
3,4,Attrited Customer,38,M,3,College,Single,$60K - $80K,Blue,34,...,3,4,12050.0,1821,10229.0,0.630,2381,40,0.481,0.151
4,5,Existing Customer,50,M,0,Uneducated,Married,$60K - $80K,Blue,46,...,1,3,3640.0,659,2981.0,0.938,3756,70,0.842,0.181


### 1.2 Category变量的处理

统计各个变量的属性和unique values，选择Category变量

In [3]:
train_data.dtypes

ID                            int64
Attrition_Flag               object
Customer_Age                  int64
Gender                       object
Dependent_count               int64
Education_Level              object
Marital_Status               object
Income_Category              object
Card_Category                object
Months_on_book                int64
Total_Relationship_Count      int64
Months_Inactive_12_mon        int64
Contacts_Count_12_mon         int64
Credit_Limit                float64
Total_Revolving_Bal           int64
Avg_Open_To_Buy             float64
Total_Amt_Chng_Q4_Q1        float64
Total_Trans_Amt               int64
Total_Trans_Ct                int64
Total_Ct_Chng_Q4_Q1         float64
Avg_Utilization_Ratio       float64
dtype: object

In [4]:
train_data.nunique()

ID                          1000
Attrition_Flag                 2
Customer_Age                  40
Gender                         2
Dependent_count                6
Education_Level                7
Marital_Status                 4
Income_Category                6
Card_Category                  4
Months_on_book                42
Total_Relationship_Count       6
Months_Inactive_12_mon         7
Contacts_Count_12_mon          7
Credit_Limit                 857
Total_Revolving_Bal          604
Avg_Open_To_Buy              911
Total_Amt_Chng_Q4_Q1         522
Total_Trans_Amt              889
Total_Trans_Ct               110
Total_Ct_Chng_Q4_Q1          443
Avg_Utilization_Ratio        502
dtype: int64

因此，我们可以选择
```Attrition_Flag, Gender, Dependent_count, Education_Level, Marital_Status, Income_Category, Card_Category, Total_Relationship_Count, Months_Inactive_12_mon, Contacts_Count_12_mon```
作为Category变量，其他列作为Numerical变量. 

在处理Category类型变量时，我们对每个类别标签用整数进行编码，在预测时进行解码.  

## 贝叶斯分类器

In [5]:
class Bayes:
    """
    class Bayes，贝叶斯分类器
     |  Bayes(method, lambda_)
     |  贝叶斯分类器，可指定朴素贝叶斯，'SODE', 'AODE', 'WAODE'半朴素贝叶斯算法\n
     |
     |  Parameters
     |  ----------
     |  method : str
     |      贝叶斯分类器所使用的算法
     |  lambda_ : float
     |      Laplace修正系数
     |  K : int
     |      待分类变迁类别数
     |  N : int
     |      训练模型所使用的样本量
     |  priori : dict
     |      存储先验概率的字典
     |  likelihood : dict
     |      存储似然概率的字典
     |  
     |  Attributes
     |  ----------
     |  fit : None
     |      给定训练样本和真实标签，计算先验概率和似然概率并存储
     |  predict : pd.Series
     |      给定数据X，对目标变量的类别进行预测
     |  accuracy : float
     |      给定测试集X_test及对应得标签y_test，计算基于模型得预测准确率
     |  __encode : pd.DataFrame
     |      特征编码函数，用于进行数据类型转换，不可外部调用
     |  __encodey : pd.Series
     |      标签编码函数，用于进行数据类型转换，不可外部调用
     |
    """
    def __init__(self, method : str = 'naive', lambda_ : float = 1) -> None:
        """
        Bayes分类器构造函数，用于设定分类器算法，Laplace修正系数\n

        Parameters
        ----------
        method : str, optional
            字符串变量，可取值['naive', 'SODE', 'AODE', 'WAODE‘]\n
            用于指定Bayes分类器所使用的算法\n
            'naive': 朴素贝叶斯算法\n
            'SODE': 超父半朴素贝叶斯算法\n
                若设置method='SODE'，还需要在调用训练函数fit时指定超父列索引\n
            'AODE': 平均独依赖估计算法\n
            'WAODE': 加权平均独依赖估计算法\n
                若设置method='WAODE'，还需要在调用训练函数fit时指定各列特征的权重\n
            默认值为'naive'，即使用朴素贝叶斯分类算法\n
        lambda_ : float, optional
            Laplace修正系数，取值大于等于0的实数\n
            默认值欸1，即Laplace平滑\n
        """
        self.method = method
        self.priori = {}
        self.likelihood = {}
        self.numerical_cols = []
        self.columns = None
        self.lambda_ = lambda_ # Laplace修正系数
        self.K = None # 预测类别个数
        self.N = None # 训练集样本数
        self.sp = None # 超父
        self.sp_index = None # 超父索引
        self.sp_count = None # 统计超父属性下各类别的样本数量
        self.sp_list = None # 超夫索引列表，用于'AODE','WODE'
        self.sp_model_list = None # 利用每一个超夫构造的'SODE'模型
        self.sp_model_weight = None # 'WODE'模型的权重
        self.m_thres = 30 # AODE对样本量的限制
    
    def fit(self, X : pd.Series, y : pd.Series, numerical_cols : list = None, super_parent : int = None, m_thres : int=30) -> None:
        """
        训练函数，接收训练集X和对应的标签y\n
        按照贝叶斯公式计算先验概率和似然概率，并储存在priori和likelihood字典中，便于预测使用\n

        Parameters
        ----------
        X : pd.DataFrame 或 np.ndarray
            模型预测的输入，作为数据特征，可以是pd.DataFram类型或np.ndarray类型\n
            如果是np.ndarray类型，则会调用self.__encode函数将输入编码为pd.DataFram类型\n
        y : pd.Series 或 np.ndarray
            模型预测的真实标签，作为label，可以是pd.Series类型或np.ndarray类型\n
            如果是np.ndarray类型，则会调用self.__encodey函数将输入编码为pd.Series类型\n
        numerical_cols : list, optional
            数值类特征的列索引，用于区分数值型特征和类别型特征\n
            如果numerical_cols=None，则默认所有特征都是类别型变量\n
        super_parent : int
            如果模型使用'SODE'方法，则需要设置超父所在的索引\n
            此项仅在模型使用‘SODE’方法时有效\n
        m_thres : int, optional
            'AODE'方法中限制的样本量\n
            仅当超父在某个属性下取值的样本量超过m_thres时，才会参与概率的计算\n
        """
        if type(X) == np.ndarray:
            X = self.__encode(X.copy())
        if type(y) == np.ndarray:
            y = self.__encodey(y.copy())
        # 添加numerical columns
        self.numerical_cols = numerical_cols
        # 带分类类别个数
        self.K = len(y.value_counts())
        # 保存总样本量
        self.N = len(y)
        # 添加自变量的列名
        self.columns = list(X)
        if self.method == 'naive':
            # 计算先验概率
            self.priori = dict((y.value_counts() + self.lambda_)/ (len(y) + self.K*self.lambda_))
            # 计算似然
            for i,col in enumerate(X.columns):
                if i in numerical_cols:
                    self.likelihood[col] = {}
                    for key in self.priori:
                        mu = X.loc[(y==key),col].mean()
                        sigma = X.loc[(y==key),col].std()
                        self.likelihood[col][key] = {}
                        self.likelihood[col][key]['mu'] = mu
                        self.likelihood[col][key]['sigma'] = sigma + 1e-10
                else:
                    self.likelihood[col] = {}
                    possible_values = list(X[col].unique())
                    for key in self.priori:
                        self.likelihood[col][key] = {}
                        for value in possible_values:
                            self.likelihood[col][key][value] = ((X[col]==value) & (y==key)).sum() / (y==key).sum()
        if self.method == 'SODE':
            # 计算超父先验概率
            assert super_parent != None
            self.sp = self.columns[super_parent]
            self.sp_index = super_parent
            # 统计超父每个属性的取值个数
            self.sp_count = X[self.sp].value_counts()
            for y_value in y.unique():
                self.priori[y_value] = {}
                for sp_value in X[self.sp].unique():
                    self.priori[y_value][sp_value] = ((y==y_value) & (X[self.sp]==sp_value)).sum() / self.N
            # 计算超父似然函数
            for i,col in enumerate(X.columns):
                if i in self.numerical_cols:
                    self.likelihood[col] = {}
                    for key in self.priori:
                        self.likelihood[col][key] = {}
                        for sp_value in self.priori[key]:
                            mu = X.loc[(y==key) & (X[self.sp]==sp_value),col].mean()
                            sigma = X.loc[(y==key) & (X[self.sp]==sp_value),col].std()
                            self.likelihood[col][key][sp_value] = {}
                            self.likelihood[col][key][sp_value]['mu'] = mu
                            self.likelihood[col][key][sp_value]['sigma'] = sigma + 1e-10
                else:
                    self.likelihood[col] = {}
                    possible_values = list(X[col].unique())
                    for key in self.priori:
                        self.likelihood[col][key] = {}
                        for sp_value in self.priori[key]:
                            self.likelihood[col][key][sp_value] = {}
                            for value in possible_values:
                                # 判断满足所有条件的集合是否是空集
                                if ((y==key) & (X[self.sp]==sp_value) & (X[col]==value)).any():
                                    self.likelihood[col][key][sp_value][value] = \
                                        ((y==key) & (X[self.sp]==sp_value) & (X[col]==value)).sum() \
                                            /  ((y==key) & (X[self.sp]==sp_value)).sum()
                                else:
                                    self.likelihood[col][key][sp_value][value] = 0
        # AODE方法和wode方法
        if self.method == 'AODE' or self.method == 'WODE':
            # 超父列表
            self.m_thres = m_thres
            self.sp_list = []
            for i in range(len(self.columns)):
                if i not in self.numerical_cols:
                    self.sp_list.append(i)
            self.sp_model_list = []
            for sp in self.sp_list:
                model = Bayes(method='SODE')
                model.fit(X,y,self.numerical_cols,super_parent=sp)
                self.sp_model_list.append(model)
            # 分别计算权重
            if self.method == 'AODE':
                self.sp_model_weight = np.array([1 for i in range(len(self.sp_list))],dtype=np.float64).reshape(-1,1)
            else:
                weight = []
                for i in self.sp_list:
                    weight.append(self.__Mutal_Imformation(X[self.columns[i]],y))
                weight = np.array(weight,dtype=np.float64).reshape(-1,1)
                weight /= weight.sum()
                self.sp_model_weight = weight

    def predict(self, X : pd.Series) -> pd.Series:
        """
        给定数据X，对目标变量的类别进行预测

        Parameters
        ----------
        X : pd.DataFrame 或 np.ndarray
            模型预测的输入，可以是pd.DataFram类型或np.ndarray类型\n
            如果是np.ndarray类型，则会调用self.__encode函数将输入编码为pd.DataFram类型\n
        
        Return
        ----------
        y_pred : pd.Series
            基于输入X，对目标变量得类别得预测
        """
        if type(X) == np.ndarray:
            X = self.__encode(X.copy())
        y_pred = []
        # 朴素贝叶斯方法
        if self.method == 'naive':
            for x in X.values:
                each_y_pred = {}
                for key in self.priori:
                    p_temp = self.priori[key]
                    for i,col in enumerate(self.columns):
                        if i in self.numerical_cols:
                            p_temp *= stats.norm.pdf(x[i],loc=self.likelihood[col][key]['mu'],scale=self.likelihood[col][key]['sigma'])
                        else:
                            # 如果测试集中出现了未知取值，则进行Laplace修正
                            if x[i] not in self.likelihood[col][key]:
                                num_of_values = len(self.likelihood[col][key].keys())+1
                                for value in self.likelihood[col][key]:
                                    numerator = self.likelihood[col][key][value]*self.N*self.priori[key] + self.lambda_
                                    denominator = self.N*self.priori[key] + num_of_values*self.lambda_
                                    self.likelihood[col][key][value] = numerator / denominator
                                self.likelihood[col][key][x[i]] = 1 / denominator
                            p_temp *= self.likelihood[col][key][x[i]]
                    each_y_pred[key] = p_temp
                each_y_pred = pd.Series(each_y_pred)
                y_pred.append(each_y_pred.index[each_y_pred.argmax()])
        # SODE方法
        if self.method == 'SODE':
            for x in X.values:
                each_y_pred = {}
                for key in self.priori:
                    p_temp = self.priori[key][x[self.sp_index]]
                    for i,col in enumerate(self.columns):
                        if i in self.numerical_cols:
                            p_temp *= stats.norm.pdf(x[i],loc=self.likelihood[col][key][x[self.sp_index]]['mu'],\
                                                          scale=self.likelihood[col][key][x[self.sp_index]]['sigma'])
                        else:
                            p_temp *= self.likelihood[col][key][x[self.sp_index]][x[i]]
                    each_y_pred[key] = p_temp
                each_y_pred = pd.Series(each_y_pred)
                y_pred.append(each_y_pred.index[each_y_pred.argmax()])
        # AODE方法和WODE方法
        if self.method == 'AODE' or self.method == 'WODE':
            for x in X.values:
                each_y_pred_sp = []
                for k,sp_model in enumerate(self.sp_model_list):
                    each_y_pred = {}
                    for key in sp_model.priori:
                        # 判断是否满足样本数要求
                        if sp_model.sp_count[x[sp_model.sp_index]] >= self.m_thres:
                            p_temp = sp_model.priori[key][x[sp_model.sp_index]]
                            for i,col in enumerate(sp_model.columns):
                                if i in sp_model.numerical_cols:
                                    p_temp *= stats.norm.pdf(x[i],loc=sp_model.likelihood[col][key][x[sp_model.sp_index]]['mu'],\
                                                                  scale=sp_model.likelihood[col][key][x[sp_model.sp_index]]['sigma'])
                                else:
                                    p_temp *= sp_model.likelihood[col][key][x[sp_model.sp_index]][x[i]]
                        # 若样本量少于m_thres，则不参与计算
                        else:
                            p_temp = 0
                        each_y_pred[key] = p_temp
                    each_y_pred_sp.append(pd.Series(each_y_pred))
                each_y_pred_sp = pd.DataFrame(each_y_pred_sp)
                each_y_pred_sp = pd.Series((self.sp_model_weight*each_y_pred_sp.values).sum(axis=0),index=each_y_pred_sp.columns)
                y_pred.append(each_y_pred_sp.index[each_y_pred_sp.argmax()])
        return y_pred
    
    def accuracy(self, X : pd.DataFrame, y_true : pd.Series) -> float:
        """
        给定测试集X_test及对应得标签y_test，计算基于模型得预测准确率

        Parameters
        ----------
        X : pd.DataFrame 或 np.ndarray
            测试样本集合\n
            如果是np.ndarray类型，则会调用self.__encode函数将输入编码为pd.DataFram类型\n
        y_true : pd.Series 或 np.ndarray
            测试样本集的正确标签\n
            如果是np.ndarray类型，则会调用self.__encodey函数将输入编码为pd.Series类型\n

        Return
        ----------
        acc : float
            基于输入测试样本X和真实标签y的模型预测准确率
        """
        if type(X) == np.ndarray:
            X = self.__encode(X.copy())
        if type(y_true) == np.ndarray:
            y_true = self.__encodey(y_true.copy())
        y_pred = self.predict(X)
        return (y_pred == y_true).sum() / len(y_true)
    
    def __encode(self, X : np.ndarray) -> np.ndarray:
        """
        特征编码函数，用于数据类型转换
        """
        if self.columns == None:
            self.columns = ['col_'+str(i+1) for i in range(X.shape[1])]
        return pd.DataFrame(X,columns=self.columns)
    
    def __encode(self, y : np.ndarray) -> pd.Series:
        """
        标签编码函数，用于数据类型转换
        """
        return pd.Series(y)
    
    def __entropy(self, X : pd.Series) -> float:
        p = X.value_counts() / len(X)
        return -(p*np.log(p)).sum()
    
    def __relative_entropy(self, X : pd.Series, Y : pd.Series) -> float:
        Y_values = Y.unique()
        X_values = X.unique()
        H = Y.value_counts()
        for y in Y_values:
            h = 0
            for x in X_values:
                if ((X==x)&(Y==y)).any():
                    p = ((X==x)&(Y==y)).sum() / (Y==y).sum()
                    h += p*np.log(p)
                else:
                    h += 0
            H[y] = -h
        Py = Y.value_counts() / len(Y)
        return (Py*H).sum()
    
    def __Mutal_Imformation(self, X : pd.Series, Y : pd.Series):
        return self.__entropy(X) - self.__relative_entropy(X,Y)

### 构建测试函数，统计正确率

In [6]:
def test(col : str, method : str, super_parent : int=None, m_thres : int=30):
    numerical_columns = [
        'Customer_Age','Months_on_book','Credit_Limit','Total_Revolving_Bal',
        'Avg_Open_To_Buy','Total_Amt_Chng_Q4_Q1','Total_Trans_Amt','Total_Trans_Ct',
        'Total_Ct_Chng_Q4_Q1','Avg_Utilization_Ratio']
    # 生成训练集和测试集
    X_train = train_data.drop(['ID',col],axis=1)
    X_test = test_data.drop(['CLIENTNUM',col],axis=1)
    y_train = train_data[col]
    y_test = test_data[col]
    # 确定数值变量所在得索引
    num_cols_index = []
    for i,c in enumerate(X_train):
        if c in numerical_columns:
            num_cols_index.append(i)
    # 创建模型
    # 训练和预测
    model = Bayes(method=method)
    model.fit(X_train,y_train,numerical_cols=num_cols_index,super_parent=super_parent,m_thres=m_thres)
    acc = model.accuracy(X_test,y_test)
    print("(%s) Predicted Column: %-25s\t Accuracy: %.4f"%(method,col,acc))

## NAIVE

In [7]:
category_columns = [
    'Attrition_Flag','Gender','Dependent_count','Education_Level',
    'Marital_Status','Income_Category','Card_Category','Total_Relationship_Count',
    'Months_Inactive_12_mon','Contacts_Count_12_mon']
for col in category_columns:
    test(col,method='naive')

(naive) Predicted Column: Attrition_Flag           	 Accuracy: 0.8750
(naive) Predicted Column: Gender                   	 Accuracy: 0.8600
(naive) Predicted Column: Dependent_count          	 Accuracy: 0.3100
(naive) Predicted Column: Education_Level          	 Accuracy: 0.3000
(naive) Predicted Column: Marital_Status           	 Accuracy: 0.4700
(naive) Predicted Column: Income_Category          	 Accuracy: 0.4900
(naive) Predicted Column: Card_Category            	 Accuracy: 0.8300
(naive) Predicted Column: Total_Relationship_Count 	 Accuracy: 0.2650
(naive) Predicted Column: Months_Inactive_12_mon   	 Accuracy: 0.3750
(naive) Predicted Column: Contacts_Count_12_mon    	 Accuracy: 0.2600


## AODE

In [8]:
for col in category_columns:
    test(col,method='AODE')

(AODE) Predicted Column: Attrition_Flag           	 Accuracy: 0.8650
(AODE) Predicted Column: Gender                   	 Accuracy: 0.8450
(AODE) Predicted Column: Dependent_count          	 Accuracy: 0.2950
(AODE) Predicted Column: Education_Level          	 Accuracy: 0.2500
(AODE) Predicted Column: Marital_Status           	 Accuracy: 0.4250
(AODE) Predicted Column: Income_Category          	 Accuracy: 0.5200
(AODE) Predicted Column: Card_Category            	 Accuracy: 0.8800
(AODE) Predicted Column: Total_Relationship_Count 	 Accuracy: 0.2500
(AODE) Predicted Column: Months_Inactive_12_mon   	 Accuracy: 0.3550
(AODE) Predicted Column: Contacts_Count_12_mon    	 Accuracy: 0.2150


## WODE

In [9]:
for col in category_columns:
    test(col,method='WODE')

(WODE) Predicted Column: Attrition_Flag           	 Accuracy: 0.8700
(WODE) Predicted Column: Gender                   	 Accuracy: 0.8400
(WODE) Predicted Column: Dependent_count          	 Accuracy: 0.2800
(WODE) Predicted Column: Education_Level          	 Accuracy: 0.2300
(WODE) Predicted Column: Marital_Status           	 Accuracy: 0.4300
(WODE) Predicted Column: Income_Category          	 Accuracy: 0.5000
(WODE) Predicted Column: Card_Category            	 Accuracy: 0.8850
(WODE) Predicted Column: Total_Relationship_Count 	 Accuracy: 0.2350
(WODE) Predicted Column: Months_Inactive_12_mon   	 Accuracy: 0.3600
(WODE) Predicted Column: Contacts_Count_12_mon    	 Accuracy: 0.2100
